In [1]:
import csv
from math import floor, log, log10
import requests
from io import StringIO
from fips_data import population_for_fips

url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'

In [2]:
csv_data = requests.get(url)

In [3]:
csv_reader = csv.reader(StringIO(csv_data.text))
next(csv_reader) # remove headers

['date', 'county', 'state', 'fips', 'cases', 'deaths']

In [4]:

def get_default_obj(fips, state):
    return {
        'fips': fips,
        'recent': {
            'cases': [],
            'deaths': []
        },
        'new': {
            'cases': [],
            'deaths': []
        },
        'totals': {
            'cases': [],
            'deaths': []
        },
    }

def per_10000(number, county):
    population = population_for_fips(county['fips'])
    if population > 0:
        return 10000 * number / population
    return 0

def get_new(key, county):
    totals = county['totals'][key]
    return totals[-1] - totals[-2] if len(totals) > 1 else totals[-1]

def get_recent(key, county, days=7):
    new = county['new'][key];
    return sum(new[-days:]) / days

counties_json = {}
for row in csv_reader:
    date,county,state,fips,cases,deaths = row
    locale = f'{county}, {state}'

    if locale == 'New York City, New York':
        fips = ['36005', '36047', '36061', '36081', '36085']
    elif locale == 'Kansas City, Missouri':
        fips = '29KCM'

    if not locale in counties_json:
        counties_json[locale] = get_default_obj(fips)

    county = counties_json[locale]

    totals = county['totals']
    totals['cases'].append(per_10000(int(cases), county))
    totals['deaths'].append(per_10000(int(deaths), county))

    new = county['new']
    new['cases'].append(get_new(key='cases', county=county))
    new['deaths'].append(get_new(key='deaths', county=county))

    recent = county['recent']
    recent['cases'].append(get_recent(key='cases', county=county))
    recent['deaths'].append(get_recent(key='deaths', county=county))

In [6]:
sorted_locales = list(counties_json.keys())
# print(counties_json)
# sorted_locales.sort(key = lambda locale : counties_json[locale]['recent_new_death_rate'], reverse=True)
# print(sorted_locales[4])
print(counties_json['Kansas City, Missouri'])
# print(counties_json.keys())

{'fips': '29KCM', 'recent': {'cases': [0.03393294736490869, 0.03676069297865109, 0.03676069297865109, 0.050899421047363044, 0.08483236841227174, 0.14421502630086194, 0.18097571927951303, 0.18663121050699782, 0.22904739471313368, 0.2516693596230728, 0.2318751403268761, 0.2516693596230728, 0.21773641225816415, 0.21208092103067933, 0.21208092103067935, 0.2205641578719065, 0.20642542980319453, 0.31670750873914777, 0.2827745613742391, 0.3421572192628293, 0.3336739824216021, 0.3676069297865108, 0.35629594733154135, 0.37609016662773814, 0.33084623680785985, 0.3591236929452837, 0.3449849648765717, 0.3647791841727685, 0.33367398242160223, 0.3223629999666326, 0.31105201751166295, 0.2573248508505574, 0.2601525964643, 0.21773641225816423, 0.2120809210306793, 0.20076993857570966, 0.20925317541693694, 0.2120809210306793, 0.22621964909939152, 0.20642542980319462, 0.203597684189452, 0.23753063155436088, 0.2997410350566935, 0.31387976312540544, 0.3280184911941174, 0.4496115525850402, 0.4863722455636911